# GOAL 

Predicting adoption churn in PySpark

Jar dependencies , also be careful of version dependencies

* hadoop-aws
* aws-java-sdk
* aws-java-sdk-core
* aws-java-sdk-s3

In [1]:
import pandas as pd
import numpy as np 

In [2]:
import findspark
findspark.init('/Users/spurushe/spark/spark-3.0.0-preview2-bin-hadoop2.7')

In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as pyf
from pyspark.sql.types import *
import functools

In [4]:
spark = SparkSession\
        .builder\
        .master("local[4]") \
        .appName("churn_pred") \
        .getOrCreate()

In [6]:
# raw_df = spark.createDataFrame(
#     [
#         (10, 100, 'Hello'),
#         (10000, 1000000, 'Shweta'),
        
#     ],
#     ["Column1", "Column2", "Column3"] # column names
# )

# raw_df.show()

+-------+-------+-------+
|Column1|Column2|Column3|
+-------+-------+-------+
|     10|    100|  Hello|
|  10000|1000000| Shweta|
+-------+-------+-------+



In [5]:
spark

<br/>
<br/>
<br/>


DATA IMPORT FROM S3

Important NOTE  
ref: https://www.philipphoffmann.de/post/spark-shell-s3a-support/
<br/>

* Out of the box Spark distribution does not include support for S3 
* You can add it by adding the required jars to spark-3.0.0-preview2-bin-hadoop2.7.jars/

<br/>

1. Choose hadoop-aws version depending on the version of hadoop files in your spark/jars folder here it was 2.7.4 for Spark 3.0.0 preview release 2.0 
2. Run in terminal 
`cd spark/spark-3.0.0-preview2-bin-hadoop2.7/jars/`
`curl -L -O https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/2.7.4/hadoop-aws-2.7.4.jar`
3. Go to the maven artifact page for hadoop-aws-2.7.4 (https://mvnrepository.com/artifact/org.apache.hadoop/hadoop-aws/2.7.4)
4. Look for the Compile dependencies > com.amazonaws » aws-java-sdk >> Version. This is the version of aws-java-sdk against which hadoop-aws-2.7.4.jar was built. So this version is very 
important. If you choose any other aws-java-sdk version, THINGS WILL BREAK. In this case it was aws-java-sdk version 1.7.4. 
5. Using the aws-java-sdk version go fetch aws-java-sdk   
`curl -L -O https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk/1.7.4/aws-java-sdk-1.7.4.jar
`
6. Similarily also fetch aws-java-sdk-core and aws-java-sdk-s3 with the same versions   
`curl -L -O https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-core/1.7.4/aws-java-sdk-core-1.7.4.jar
 curl -L -O https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-core-s3/1.7.4/aws-java-sdk-core-s3-1.7.4.jar
`

<br/>
<br/>
<br/>
<br/>





In [6]:
bucket_name = '*'
data_path = '*'

In [7]:
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", "*")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "*")
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.amazonaws.com")

In [8]:
df = spark.read.csv(f's3a://{bucket_name}/{data_path}/dummy_tweets.csv')

In [9]:
df.show()

+--------------------+
|                 _c0|
+--------------------+
|This tweet is abo...|
|This tweet is abo...|
|This tweet is abo...|
|This tweet is abo...|
|This tweet is abo...|
+--------------------+



In [15]:
df = spark.read.format('csv').load(f's3a://{bucket_name}/{data_path}/*.csv', header= True, inferSchema=True)

In [30]:
spark.stop()